In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *

qm = QHJMall(MALL_KEY)

In [2]:

save_dir = r"D:\Downloads\QHJ_MALL"
save = SaveXl(save_dir)



In [3]:
# 正式
# conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

# 测试
conn = redis.Redis(**REDIS_TEST_DIC)


### 获取订单号的参数

```python

qm = QHJMall(MALL_KEY)
qm.get_order(order_type,**kargs)


# 请求类型 0 表示所有订单 1 未发货  2 已发货 3 已完成 4 申请退款 5 退款成功  10 未支付
"order_status": kargs.get("order_status", 0),
"order_time": kargs.get("order_time", ""), # today yesterday week month
"order_id": kargs.get("order_id", ""), # 订单号
"order_type": order_type,  # original free
"page": kargs.get("page", 1),  #

```


In [4]:
sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

free_goods = sku_goods[sku_goods['goods_type']=='free']


In [5]:
order_kargs = {
    "order_status":1,
    "order_time":"week"
}
original_data = qm.get_original_orders(sku_goods,order_kargs)
free_data = qm.get_free_orders(free_goods,order_kargs)


data = pd.concat([free_data,original_data],sort = True)
l = ['订单号', '订单状态','商品ID', '下单时间', '支付时间','支付金额',

 '商品名', '规格', '单位', '单价', '数量', '备注',
    
'收件人', '联系方式', '收货地址','goods_type','发货商ID','redis_pos']

data_r = data[l]

data_r.loc[:,'name'] = "ORDER:NEW:" + data_r['发货商ID'] + ":" +data_r['redis_pos']
data_r

请求 {'order_status': 1, 'order_time': 'week', 'order_id': '', 'order_type': 'original', 'page': 1}
请求 {'order_status': 1, 'order_time': 'week', 'order_id': '', 'order_type': 'free', 'page': 1}
请求 {'order_status': 1, 'order_time': 'week', 'order_id': '', 'order_type': 'free', 'page': 2}
请求 {'order_status': 1, 'order_time': 'week', 'order_id': '', 'order_type': 'free', 'page': 3}
请求 {'order_status': 1, 'order_time': 'week', 'order_id': '', 'order_type': 'free', 'page': 4}


c:\users\qhj01\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\qhj01\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,订单号,订单状态,商品ID,下单时间,支付时间,支付金额,商品名,规格,单位,单价,数量,备注,收件人,联系方式,收货地址,goods_type,发货商ID,redis_pos,name
0,8171817913572174,未发货,S0003P0001C0001P01,2019-12-12 20:12:38,2019-12-12 20:12:45,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138,1,,桃桃,13875918846,湖南省长沙市芙蓉区解放东路300号华天大酒店5楼,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
1,2758468066946996,未发货,S0003P0001C0001P01,2019-12-12 20:12:00,2019-12-12 20:12:10,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138,1,,周美蓉,15274927704,湖南省衡阳市石鼓区湖南工商职业学院,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
2,0501250902978832,未发货,S0003P0001C0003P01,2019-12-12 20:12:37,2019-12-12 20:12:46,0,三金护肤面膜 维稳修护 强化屏障(护肤),5片*25g,盒,138,1,,杨先生,13307364056,湖南省长沙市岳麓区湖南省长沙市岳麓区湖南涉外经济学院,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
3,2163778663397136,未发货,S0003P0001C0001P01,2019-12-12 19:12:39,2019-12-12 19:12:47,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138,1,,李丽娜,15273293792,湖南省长沙市芙蓉区解放中路86号省住建厅2304,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
4,3960831208197543,未发货,S0003P0001C0002P01,2019-12-12 19:12:04,2019-12-12 19:12:19,0,三金美肤面膜 美白功效 健康肤色(美肤),5片*25g,盒,138,1,,龙哥,18670022695,湖南省长沙市芙蓉区旭日东升b座502,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
5,1554653366651438,未发货,S0003P0001C0001P01,2019-12-12 19:12:43,2019-12-12 19:12:54,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138,1,,龙哥,18670022695,湖南省长沙市芙蓉区旭日东升b座502,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
6,9076677957182044,未发货,S0003P0001C0001P01,2019-12-12 19:12:00,2019-12-12 19:12:09,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138,1,,曹阳,18173190551,湖南省长沙市雨花区城南中路75号,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
7,1015787224552650,未发货,S0003P0001C0003P01,2019-12-12 19:12:43,2019-12-12 19:12:01,0,三金护肤面膜 维稳修护 强化屏障(护肤),5片*25g,盒,138,1,,杨先生,13307364056,湖南省长沙市岳麓区湖南省长沙市岳麓区湖南涉外经济学院,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
8,0149891054111246,未发货,S0003P0001C0004P01,2019-12-12 19:12:22,2019-12-12 19:12:32,0,三金水光针(水光针),10ml/支,盒,109,1,,杨先生,13307364056,湖南省长沙市岳麓区湖南省长沙市岳麓区湖南涉外经济学院,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12
9,2684946144160119,未发货,S0003P0001C0001P01,2019-12-12 19:12:17,2019-12-12 19:12:24,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138,1,,杨先生,13307364056,湖南省长沙市岳麓区湖南省长沙市岳麓区湖南涉外经济学院,free,S0003,2019:12:12,ORDER:NEW:S0003:2019:12:12


In [6]:
stop

NameError: name 'stop' is not defined

In [ ]:
import datetime
class OrderEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        
        if isinstance(obj, datetime.datetime):  
            return obj.strftime('%Y-%m-%d %H:%M:%S')  
        else:
            return super(MyEncoder, self).default(obj)



In [ ]:
order_kargs = {
    "order_status":1,
    "order_time":"week"
}
original_data = qm.get_original_orders(sku_goods,order_kargs)
free_data = qm.get_free_orders(free_goods,order_kargs)


data = pd.concat([free_data,original_data],sort = True)
l = ['订单号', '订单状态','商品ID', '下单时间', '支付时间','支付金额',

 '商品名', '规格', '单位', '单价', '数量', '备注',
    
'收件人', '联系方式', '收货地址','goods_type','发货商ID','redis_pos']

data_r = data[l]

data_r.loc[:,'name'] = "ORDER:NEW:" + data_r['发货商ID'] + ":" +data_r['redis_pos']

In [ ]:


def uprow(row):

    js = json.dumps(row.to_dict(),cls=OrderEncoder,ensure_ascii=False)
    order_num = row['订单号']
    spp_num = row['发货商ID']
    # 判断是否在已经 发过此物
    if not conn.sismember('ORDER:NEW:%s' %spp_num,order_num):
        conn.hset(row['name'] ,order_num,js)
        conn.sadd("ORDER:NEW:TEMP:%s" %spp_num,order_num)
    else:
        row['订单号'] == None
data_r.apply(uprow,axis=1)

save(data_r,"订单")
%run D:\往期\QHJ\ZERO\app\process_order.py


In [ ]:
stop

In [ ]:
spp_path =  os.path.join(r"D:\奇货居\素材\商城图片素材","发货商详情.xlsx")
spp_data = pd.read_excel(spp_path)

In [ ]:
spp_data

In [ ]:
stop

In [ ]:
d = dict(zip(spp_data['发货商ID'],spp_data['发货商'],))

d

In [ ]:
conn.keys('ORDER:NEW:TEMP*')

In [8]:
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

conn.keys('ORDER:NEW:TEMP*')
src_new = 'ORDER:NEW:TEMP:S0003'
dst_new = src_new.replace(":TEMP","")
conn.sunionstore( dst_new,dst_new,src_new)
conn.delete(src_new)

1

In [ ]:
# conn.sunionstore('ORDER:NEW',"ORDER:NEW_TEMP",'ORDER:NEW')

# conn.sadd("ORDER:NEW","zero")
# conn.sadd("ORDER:WB","zero")
# conn.sadd("ORDER:NEW_TEMP","zero")